In [ ]:
# !cp CPPred/bin/feamodule/CTD.py CTD.py
# !cp CPPred/bin/feamodule/ProtParam.py ProtParam.py
# !cp CPPred/bin/feamodule/ORF.py ORF.py
# !cp CPPred/bin/feamodule/fickett.py fickett.py
# !cp CPPred/bin/feamodule/FrameKmer.py FrameKmer.py
# !cp CPPred/bin/feamodule/ORF_length.py ORF_length.py

In [ ]:
!pip install Bio
!pip install biopython==1.75 --user

In [1]:
import os,sys

hex_file="CPPred/Hexamer/Human_Hexamer.tsv"
seq_file="CPPred/data/Human_coding_RNA_test.fa"

In [2]:
lines = open(hex_file).readlines()
lines[:5]

['hexamer\tcoding\tnoncoding\n',
 'GAACGT\t0.000125472613061\t6.29880112819e-05\n',
 'CTTCTT\t0.000299994100152\t0.000441227130881\n',
 'CACCCT\t0.000259819200042\t0.000355221278439\n',
 'GAACGG\t0.000195066081257\t6.15882776978e-05\n']

In [3]:
coding={}
noncoding={}
for line in lines:
    fields = line.split()
    if fields[0] == 'hexamer':continue           # 表头
    coding[fields[0]] = float(fields[1])
    noncoding[fields[0]] =  float(fields[2])

In [4]:
import Bio.SeqIO as Seq
seqs = Seq.parse(seq_file,'fasta')

for seq in seqs:
    print(seq);
    break;

ID: gi|1238415392|ref|NM_001354394.1|
Name: gi|1238415392|ref|NM_001354394.1|
Description: gi|1238415392|ref|NM_001354394.1| Homo sapiens diphosphoinositol pentakisphosphate kinase 1 (PPIP5K1), transcript variant 20, mRNA
Number of features: 0
Seq('ACCGCAAAGAAGTCGCGCTTTGATTGGGCCGCTGTCAGGTGTCGCTGCCTGCAG...GGA', SingleLetterAlphabet())


In [ ]:
# import importlib
# importlib.reload(len)

<module 'ORF_length' from '/home/aistudio/ORF_length.py'>

In [5]:
tmp = open('test.f_svm','w')
feature = open('test.feature','w')

# species == "Human":
feature.write("\t".join(map(str,["#ID","ORF-integrity","ORF-coverage","Instability","T2","C0","PI","ORF-length","AC","T0","G0","C2","A4","G2","TG","A0","TC","G1","C3","T3","A1","GC","T1","G4","C1","G3","A3","Gravy","Hexamer","C4","AG","Fickett","A2","T4","C","G","A","T"]))+"\n")
type(feature)

_io.TextIOWrapper

In [6]:
import CTD
import ProtParam as PP
import fickett
import FrameKmer
import ORF_length as len

out_label = 1

for seq in seqs:
    seqid = seq.id
    A,T,G,C,AT,AG,AC,TG,TC,GC,A0,A1,A2,A3,A4,T0,T1,T2,T3,T4,G0,G1,G2,G3,G4,C0,C1,C2,C3,C4 = CTD.CTD(seq.seq)
    insta_fe,PI_fe,gra_fe = PP.param(seq.seq)

    fickett_fe = fickett.fickett_value(seq.seq)
    hexamer = FrameKmer.kmer_ratio(seq.seq,6,3,coding,noncoding)
    Len,Cov,inte_fe = len.len_cov(seq.seq)
    
    tem = [inte_fe,Cov,insta_fe,T2,C0,PI_fe,Len,AC,T0,G0,C2,A4,G2,TG,A0,TC,G1,C3,T3,A1,GC,T1,G4,C1,G3,A3,gra_fe,hexamer,C4,AG,fickett_fe,A2,T4,C,G,A,T]
    feature.write("\t".join(map(str,[seqid,inte_fe,Cov,insta_fe,T2,C0,PI_fe,Len,AC,T0,G0,C2,A4,G2,TG,A0,TC,G1,C3,T3,A1,GC,T1,G4,C1,G3,A3,gra_fe,hexamer,C4,AG,fickett_fe,A2,T4,C,G,A,T]))+"\n")
 
    # print (tmp)
    print (out_label)
    for label,item in enumerate(tem):
        print (tmp)
        print(str(label+1)+':'+str(item))

tmp.close()

/home/aistudio/.local/lib/python3.7/site-packages/Bio/Seq.py:182: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [ ]:
def predict(range_file,model_file):
	os.system('../libsvm-3.22/svm-scale -r '+ range_file + ' test.f_svm  > test.scaled ')
	os.system('../libsvm-3.22/svm-predict -b 1 test.scaled ' + model_file +' tmp.txt >  tmp2.txt')

	coding_poten = open('coding_potential','w')
	coding_poten.write("\t".join(map(str,["table","coding_potential"]))+"\n")

	for line in open('tmp.txt').readlines():
		if line[0]=="l":
			continue
		coding_potential=line.split(" ")[1]
		if line.split(" ")[0] == "1":
			coding_poten.write("\t".join(map(str,["coding",coding_potential]))+"\n")
		else:
			coding_poten.write("\t".join(map(str,["noncoding",coding_potential]))+"\n")

In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 